In [5]:
config_path = './results/new_design/config.yaml'
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm
import src.ASM as ASM
import src.math_tool as mt
import src.img_tool as it
from PIL import Image
import yaml
from box import Box
%load_ext autoreload
%autoreload 2
%matplotlib ipympl
time_string = datetime.now().isoformat(timespec='minutes')
################################################## all code in unit of μm
config = None
with open(config_path, 'r') as f:
    config = Box(yaml.safe_load(f))
π, λ = np.pi, 0.532
k = np.array(2*π/λ).astype(np.float32)
x_start, x_end, y_start, y_end, z_start, z_end = tuple(config.simulation.range)
Lx, Ly, Lz = x_end - x_start, y_end - y_start, z_end - z_start
Nx_ticks, Ny_ticks, Nz_ticks = tuple(config.plot.N_ticks) 
z_batch_size = eval(config.simulation.z_batch_size)
s_res = config.simulation.resolution
Nx, Ny, Nz = eval(s_res[0]), eval(s_res[1]), eval(s_res[2]) # manufactured aperture have resolution of 1000 * 1000
# xs, ys, zs = np.linspace(x_start, x_end, Nx, endpoint=False), np.linspace(y_start, y_end, Ny, endpoint=False), np.linspace(z_start, z_end, Nz, endpoint=False)
xs, ys, zs = np.linspace(x_start, x_end, Nx, endpoint=False, dtype=np.float32), np.linspace(y_start, y_end, Ny, endpoint=False, dtype=np.float32), np.linspace(z_start, z_end, Nz, endpoint=False, dtype=np.float32)
x_ticks, x_ticks_val, y_ticks, y_ticks_val, z_ticks, z_ticks_val = np.arange(0, Nx, Nx/Nx_ticks), np.round(xs[::Nx//Nx_ticks], decimals=config.plot.dp), np.arange(0, Ny, Ny/Ny_ticks), np.round(ys[::Ny//Ny_ticks], decimals=config.plot.dp), np.arange(0, Nz, Nz/Nz_ticks), np.round(zs[::Nz//Nz_ticks], decimals=config.plot.dp)
Δx, Δy, Δz = xs[1]-xs[0], ys[1]-ys[0], zs[1]-zs[0]
xs+=Δx/2; ys+=Δy/2 #center coordinate absolutely
ND = 20
Nθs = 32
D_start, D_end = 30e3, 50e3
Ds = np.linspace(D_start, D_end, ND, endpoint=False, dtype=np.float32)
ND_ticks = 5
D_ticks, D_ticks_val = np.arange(0, ND, ND/ND_ticks), np.linspace(D_start, D_end, ND_ticks, endpoint=False, dtype=np.float32)
θ0s = np.linspace(0, 2*π, Nθs, endpoint=False, dtype=np.float32)
x, y = np.meshgrid(xs, ys, indexing='ij')
def cal_distance(x, y, D): #calculate distance from set x, y, to (0, 0, D)
    '''
        x, y <2d array>
        D <1d array>
    return
        <3d array> # Nx, Ny, ND
    '''
    return np.sqrt( D**2 + mt.r(x, y, 0, 0)[..., None]**2)
print()
print(f'Nx={Nx}, Ny={Ny}, ND={ND}, Nθs={Nθs}')
print(f'float32 memory requirement: {32*Nx*Ny*ND*Nθs/8/1024**3}GB')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

Nx=512, Ny=512, ND=20, Nθs=32
float32 memory requirement: 0.625GB


In [7]:
# revolve the aperture design to produce a donut shaped focus with donut radius r_d at focal distance D
r_d = 50 # radius of donut # also try negative number
D = 25e3 # focal distance
r = (x**2 + y**2)**0.5
θ_plus = np.mod(k * ((r - r_d)**2 + D**2)**0.5, np.float32(2*π))
θ_minus = np.mod(k * ((r - (-r_d))**2 + D**2)**0.5, np.float32(2*π))
cos_θ_plus_θ0s = np.cos(θ_plus[..., None]-θ0s) # Nx, Ny, Nθs
aperture_plus = (cos_θ_plus_θ0s > 0) # aperture only selects location that have constructive contribution to θ0
cos_θ_minus_θ0s = np.cos(θ_minus[..., None]-θ0s) # Nx, Ny, Nθs
aperture_minus = (cos_θ_minus_θ0s > 0) # aperture only selects location that have constructive contribution to θ0
print(aperture_plus.shape)

(512, 512, 32)


In [28]:
save_path = './aperture/tmp'
import src.apertures as apertures
mask = apertures.block_edge_mask(512, 512, 250)
aperture_plus *= mask[..., None]
aperture_minus *= mask[..., None]
intensity_plus = aperture_plus.sum(axis=(0, 1))
intensity_minus = aperture_minus.sum(axis=(0, 1))
max_plus_idx = intensity_plus.argmax()
max_minus_idx = intensity_minus.argmax()
print(intensity_plus.min()/intensity_plus.max())
print(intensity_minus.min()/intensity_minus.max())
best_plus = aperture_plus[..., max_plus_idx]
best_minus = aperture_minus[..., max_minus_idx]

0.843170383720057
0.9608947473537048


In [29]:
it.array_2_img(best_plus, f'{save_path}/f25_donut_50_plus.bmp')
it.array_2_img(best_minus, f'{save_path}/f25_donut_50_minus.bmp')

In [30]:
pad_edge = (2048 - 1024)//2
best_plus_zoom = np.repeat(best_plus, 2, axis=0)
best_plus_zoom = np.repeat(best_plus_zoom, 2, axis=1)
best_plus_zoom_pad = it.img_pad(best_plus_zoom, (pad_edge, pad_edge, pad_edge, pad_edge), pad_val=False)
best_minus_zoom = np.repeat(best_minus, 2, axis=0)
best_minus_zoom = np.repeat(best_minus_zoom, 2, axis=1)
best_minus_zoom_pad = it.img_pad(best_minus_zoom, (pad_edge, pad_edge, pad_edge, pad_edge), pad_val=False)

In [31]:
it.array_2_img(best_plus_zoom_pad, f'{save_path}/f25_donut_50_plus_zoom_pad.bmp')
it.array_2_img(best_minus_zoom_pad, f'{save_path}/f25_donut_50_minus_zoom_pad.bmp')